In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define the finite difference methods

def forward_euler(lambda_, dx, x_max):
    """Solves dy/dx = -lambda*y using Forward Euler method."""
    x_vals = np.arange(0, x_max + dx, dx)
    y_vals = np.zeros_like(x_vals)
    y_vals[0] = 1  # Initial condition y(0) = 1

    for i in range(1, len(x_vals)):
        y_vals[i] = y_vals[i-1] - lambda_ * dx * y_vals[i-1]

    exact_solution = np.exp(-lambda_ * x_vals)
    error = np.abs(y_vals - exact_solution).sum()

    return x_vals, y_vals, exact_solution, error

def central_difference_oscillator(omega, dx, x_max):
    """Solves d²y/dx² + omega² y = 0 using Central Difference."""
    x_vals = np.arange(0, x_max + dx, dx)
    y_vals = np.zeros_like(x_vals)
    y_vals[0] = 1  # Initial condition y(0) = 1
    y_vals[1] = np.cos(omega * dx)  # Approximate first step

    for i in range(2, len(x_vals)):
        y_vals[i] = 2 * y_vals[i-1] - y_vals[i-2] - omega**2 * dx**2 * y_vals[i-1]

    exact_solution = np.cos(omega * x_vals)
    error = np.abs(y_vals - exact_solution).sum()

    return x_vals, y_vals, exact_solution, error

def heat_equation(alpha, dx, dt, x_max, t_max):
    """Solves du/dt = alpha d²u/dx² using an explicit finite difference method."""
    x_vals = np.arange(0, x_max + dx, dx)
    t_vals = np.arange(0, t_max + dt, dt)
    u = np.zeros((len(t_vals), len(x_vals)))

    # Initial condition: u(x,0) = sin(pi * x)
    u[0, :] = np.sin(np.pi * x_vals)

    # Stability condition for explicit scheme: dt <= dx^2 / (2*alpha)
    if dt > dx**2 / (2 * alpha):
        print(f"Warning: dt ({dt}) is too large; numerical instability may occur.")

    # Time stepping
    for n in range(len(t_vals) - 1):
        for i in range(1, len(x_vals) - 1):
            u[n+1, i] = u[n, i] + (alpha * dt / dx**2) * (u[n, i+1] - 2 * u[n, i] + u[n, i-1])

    # Exact solution at final time step
    exact_solution = np.exp(-np.pi**2 * alpha * t_max) * np.sin(np.pi * x_vals)
    error = np.abs(u[-1, :] - exact_solution).sum()

    return x_vals, u[-1, :], exact_solution, error

# Define parameters
lambda_ = 1.0  # Decay equation parameter
omega = 2.0    # Harmonic oscillator frequency
alpha = 0.1    # Diffusivity for heat equation
dt = 0.01
t_max = 1.0
x_max = 10

dx_values = [0.5, 0.2, 0.1, 0.05, 0.01]
results = []

# Compute numerical solutions for different dx values
for dx in dx_values:
    x_fe, y_fe, exact_fe, error_fe = forward_euler(lambda_, dx, x_max)
    x_cd, y_cd, exact_cd, error_cd = central_difference_oscillator(omega, dx, x_max)
    x_h, u_h, exact_h, error_h = heat_equation(alpha, dx, dt, x_max, t_max)
    
    results.append({
        "dx": dx,
        "Error (Decay Eq.)": error_fe,
        "Error (Oscillator)": error_cd,
        "Error (Heat Eq.)": error_h
    })

# Convert results into a dataframe
df_results = pd.DataFrame(results)

# Display results
print("\nFinite Difference Errors for Different Equations:\n")
print(df_results.to_string(index=False))

# Generate consistent x values for plotting
x_fe = np.linspace(0, x_max, 100)
x_cd = np.linspace(0, x_max, 100)
x_h = np.linspace(0, x_max, 100)

# Compute exact solutions for plotting
exact_fe = np.exp(-lambda_ * x_fe)
exact_cd = np.cos(omega * x_cd)
exact_h = np.exp(-np.pi**2 * alpha * t_max) * np.sin(np.pi * x_h)

# Create figure and subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Decay Equation Plot
axes[0].plot(x_fe, exact_fe, 'k-', label="Exact Solution")
for dx in dx_values:
    x_fe, y_fe, _, _ = forward_euler(lambda_, dx, x_max)
    axes[0].plot(x_fe, y_fe, label=f"dx = {dx}")
axes[0].set_title(r"Decay Eq: $\frac{dy}{dx} = -\lambda y$ (Forward Euler)")
axes[0].set_xlabel(r"$x$")
axes[0].set_ylabel(r"$y$")
axes[0].legend()

# Oscillator Equation Plot
axes[1].plot(x_cd, exact_cd, 'k-', label="Exact Solution")
for dx in dx_values:
    x_cd, y_cd, _, _ = central_difference_oscillator(omega, dx, x_max)
    axes[1].plot(x_cd, y_cd, label=f"dx = {dx}")
axes[1].set_title(r"Oscillator Eq: $\frac{d^2y}{dx^2} + \omega^2 y = 0$")
axes[1].set_xlabel(r"$x$")
axes[1].set_ylabel(r"$y$")
axes[1].legend()

# Heat Equation Plot
axes[2].plot(x_h, exact_h, 'k-', label="Exact Solution")
for dx in dx_values:
    x_h, u_h, _, _ = heat_equation(alpha, dx, dt, x_max, t_max)
    axes[2].plot(x_h, u_h, label=f"dx = {dx}")
axes[2].set_title(r"Heat Eq: $\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}$")
axes[2].set_xlabel(r"$x$")
axes[2].set_ylabel(r"$u(x,t_{final})$")
axes[2].legend()

plt.tight_layout()
plt.show()

# Recompute results with the heat equation error correctly included
results = []

for dx in dx_values:
    x_fe, y_fe, exact_fe, error_fe = forward_euler(lambda_, dx, x_max)
    x_cd, y_cd, exact_cd, error_cd = central_difference_oscillator(omega, dx, x_max)
    x_h, u_h, exact_h, error_h = heat_equation(alpha, dx, dt, x_max, t_max)
    
    results.append({
        "dx": dx,
        "Error (Decay Eq.)": error_fe,
        "Error (Oscillator)": error_cd,
        "Error (Heat Eq.)": error_h
    })

# Convert results into a dataframe
df_results = pd.DataFrame(results)

# Compute the total error for each dx by summing the errors across all equations
df_results["Total Error"] = df_results["Error (Decay Eq.)"] + df_results["Error (Oscillator)"] + df_results["Error (Heat Eq.)"]

# Find the dx value that minimizes the total error while maintaining stability
optimal_dx = df_results.loc[df_results["Total Error"].idxmin(), "dx"]

# Display the updated table with the optimal dx highlighted
print("\nFinite Difference Errors for Different Equations (With Total Error):\n")
print(df_results.to_string(index=False))

# Print the best choice of dx
print(f"\nThe most appropriate choice of dx is **{optimal_dx}**, as it minimizes total error while maintaining numerical stability.")

In [ ]:
# Wave equation parameters
c = 1.0  # Wave speed
L = 10.0  # Domain length
T = 10.0  # Total time
dt = 0.01  # Fixed time step
dx_values_wave = [0.5, 0.2, 0.1, 0.05, 0.01]  # Different dx values

def wave_equation(dx, dt, c, L, T):
    """Solves the wave equation using explicit finite difference scheme."""
    x_vals = np.arange(0, L + dx, dx)
    t_vals = np.arange(0, T + dt, dt)
    u = np.zeros((len(t_vals), len(x_vals)))

    # Initial condition: Gaussian pulse at center
    u[0, :] = np.exp(-((x_vals - L/2) ** 2) / 0.5)

    # First time step using first-order approximation
    u[1, :] = u[0, :]

    # Explicit finite difference scheme
    for n in range(1, len(t_vals) - 1):
        for i in range(1, len(x_vals) - 1):
            u[n+1, i] = 2*u[n, i] - u[n-1, i] + (c**2 * dt**2 / dx**2) * (u[n, i+1] - 2*u[n, i] + u[n, i-1])

    return x_vals, u[-1, :]  # Return final time step solution

# Plot results for different dx values
plt.figure(figsize=(12, 6))
for dx in dx_values_wave:
    x_wave, u_wave = wave_equation(dx, dt, c, L, T)
    plt.plot(x_wave, u_wave, label=f"dx = {dx}")

plt.title(r"Overfitting in Wave Equation: $\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$")
plt.xlabel("x")
plt.ylabel("u(x, T)")
plt.legend()
plt.show()


In [ ]:
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

def heat_equation_implicit(alpha, dx, dt, x_max, t_max):
    """Solves the heat equation using the implicit backward Euler method."""
    x_vals = np.arange(0, x_max + dx, dx)
    t_vals = np.arange(0, t_max + dt, dt)
    N = len(x_vals)
    
    # Set up the tridiagonal matrix A
    r = alpha * dt / dx**2
    main_diag = (1 + 2 * r) * np.ones(N)
    off_diag = -r * np.ones(N - 1)
    A = diags([off_diag, main_diag, off_diag], [-1, 0, 1], format="csr")

    # Initial condition: sine wave
    u = np.sin(np.pi * x_vals)
    
    # Time stepping using implicit method
    for _ in range(len(t_vals) - 1):
        u = spsolve(A, u)  # Solve the linear system

    return x_vals, u

# Define dx values to observe overfitting
dx_values_implicit = [0.1, 0.05, 0.01, 0.0001]

# Plot results for different dx values
plt.figure(figsize=(12, 6))
for dx in dx_values_implicit:
    x_h, u_h = heat_equation_implicit(alpha, dx, dt, x_max, t_max)
    plt.plot(x_h, u_h, label=f"dx = {dx}")

plt.title(r"Overfitting in Implicit Heat Equation: $\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}$")
plt.xlabel("x")
plt.ylabel("u(x,t_{final})")
plt.legend()
plt.show()

In [ ]:
import time

# Measure computation time for different dx values
computation_times = []
plt.figure(figsize=(12, 6))

for dx in dx_values_implicit:
    start_time = time.time()
    x_h, u_h = heat_equation_implicit(alpha, dx, dt, x_max, t_max)
    end_time = time.time()
    computation_times.append(end_time - start_time)

    plt.plot(x_h, u_h, label=f"dx = {dx}")

plt.title(r"Overfitting in Implicit Heat Equation: $\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}$")
plt.xlabel("x")
plt.ylabel("u(x,t_{final})")
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 0.5)
plt.legend()
plt.show()

# Show computation times for different dx values
df_computation_time = pd.DataFrame({
    "dx": dx_values_implicit,
    "Computation Time (seconds)": computation_times
})

print("\nComputation Time for Different dx Values in Implicit Solver:\n")
print(df_computation_time.to_string(index=False))


In [ ]:
# Define a zoomed-in range to focus on oscillations
zoom_x_min, zoom_x_max = 0.4, 0.6  # Focus on the middle region where oscillations appear
zoom_y_min, zoom_y_max = 0.2, 0.5  # Set y-limits to highlight oscillations

# Create figure for zoomed-in oscillations
plt.figure(figsize=(12, 6))

for dx in dx_values_implicit:
    x_h, u_h = heat_equation_implicit(alpha, dx, dt, x_max, t_max)
    plt.plot(x_h, u_h, label=f"dx = {dx}")

plt.title("Zoomed-in View of Overfitting in Implicit Heat Equation")
plt.xlabel("x")
plt.ylabel("u(x,t_final)")
plt.xlim(zoom_x_min, zoom_x_max)  # Set zoomed-in x-axis range
plt.ylim(zoom_y_min, zoom_y_max)  # Set zoomed-in y-axis range
plt.legend()
plt.show()

# Compute numerical errors by comparing with an exact solution
def exact_solution_heat(x, t, alpha):
    """Exact analytical solution for the heat equation with sine wave initial condition."""
    return np.exp(-np.pi**2 * alpha * t) * np.sin(np.pi * x)

# Compute errors for different dx values
errors = []

for dx in dx_values_implicit:
    x_h, u_h = heat_equation_implicit(alpha, dx, dt, x_max, t_max)
    u_exact = exact_solution_heat(x_h, t_max, alpha)
    error = np.linalg.norm(u_h - u_exact) / np.linalg.norm(u_exact)  # Relative error
    errors.append(error)

# Create a table of numerical errors
df_errors = pd.DataFrame({
    "dx": dx_values_implicit,
    "Relative Error": errors
})

# Display the numerical error table
print("\nNumerical Errors for Different dx Values:\n")
print(df_errors.to_string(index=False))
